# Lab: Build a Book Recommendation *Agent* with One MCP Tool (Goodreads)

## What this lab is about

In this lab you will build a **book recommendation AI agent** that uses the **Model Context Protocol (MCP)** to call an external tool. The agent reads a real Goodreads dataset and decides — on its own — whether a user's question requires looking up the data or can be answered directly.

This teaches the central idea behind agentic AI: **an LLM that can choose to act**, not just generate text.

---

## How the system fits together

```
User question
      │
      ▼
┌─────────────────────┐
│   Agent (policy)    │  ← decides: call tool, or answer directly?
└────────┬────────────┘
         │  tool call (structured JSON)
         ▼
┌─────────────────────┐
│   MCP Server        │  ← exposes recommend_books() as a callable tool
└────────┬────────────┘
         │  queries
         ▼
┌─────────────────────┐
│  Goodreads CSV      │  ← the ground-truth data source
└─────────────────────┘
         │  structured results (JSON list)
         ▼
Agent formats → natural language response → User
```

There are **three distinct components** you will build:

| Component | What it does | Where in the lab |
|-----------|-------------|------------------|
| **Recommender function** | Filters/sorts the CSV | Steps 3–4 |
| **MCP Server** | Exposes that function as a network-callable tool | Steps 5–6 |
| **Agent** | Decides when to call the tool vs. answer directly | Steps 7–8 |

You will:
1. Load a trimmed Goodreads dataset (CSV)
2. Wrap a recommendation function as an **MCP tool** (`recommend_books`)
3. Run an MCP server (HTTP) that exposes that tool
4. Connect a client and verify tool calls work
5. Build a simple agent loop that decides *when* to call the tool


## Learning Objectives

By the end of this lab you will be able to:

- **Explain MCP in plain language**: it is a standard way for an agent to discover and call tools, similar to how a USB-C port standardizes how devices connect — any MCP agent can plug into any MCP server.
- **Build and run an MCP server** that exposes a single Python function as a callable tool over HTTP.
- **Call a tool from a client** and interpret the structured JSON results.
- **Implement a minimal agent policy**: the logic that decides whether to call the tool or answer a question directly from the agent's own knowledge.
- **Articulate the difference** between a *hallucinated* response (the agent makes up book ratings) and a *grounded* response (the agent reads real data from the CSV).


## Before You Start

### 1) The Goodreads CSV

This lab uses a cleaned Goodreads dataset. The CSV must have columns for title, author, and average rating (genre/shelf columns are optional but improve recommendations). A compatible file is provided with the lab materials.

### 2) Upload the CSV to Databricks

In **Databricks Free Edition**, upload the file via the UI:

1. Click **Catalog** in the left sidebar → **Add data** → **Upload files**
2. Select your CSV and upload it to a Volume (e.g., `/Volumes/workspace/goodreads_lab/goodreads/`)
3. Copy the resulting file path — you will paste it into `DATA_PATH` in Step 1

In the next step you will set `DATA_PATH` to the location of your CSV, and `SAMPLE_ROWS` to control how many rows to load (smaller = faster iteration).


## Step 0 — Install Dependencies

Before any code can run, we need two Python packages:

- **`fastmcp`** — a Python library that makes it easy to create an MCP server. It handles the HTTP transport, tool registration, and the MCP protocol framing so you don't have to write that boilerplate yourself.
- **`pandas`** — for loading and querying the Goodreads CSV. It gives us the DataFrame that the recommender logic will filter and sort.

The `%pip install` magic works inside Databricks notebooks and installs packages into the current cluster environment. After installing new packages, Databricks requires a Python interpreter restart (the next cell handles this) so the newly installed modules are importable.

> **Why `fastmcp` and not the raw MCP SDK?** `fastmcp` is a high-level wrapper that lets you turn a plain Python function into an MCP tool with a single decorator (`@mcp.tool`). It is the fastest way to understand MCP without drowning in protocol details.


In [0]:
%pip install -q fastmcp pandas

In [0]:
dbutils.library.restartPython()

## Step 1 — Point to Your Goodreads CSV

This cell sets two configuration constants used by every subsequent step:

- **`DATA_PATH`** — the full path to your CSV inside Databricks. Update this to wherever you uploaded the file in the prerequisite step. Databricks Volumes paths look like `/Volumes/<catalog>/<schema>/<volume>/<filename>.csv`.
- **`SAMPLE_ROWS`** — how many rows to read from the CSV. The full Goodreads dataset can be large; loading 20,000 rows is fast and more than enough for this lab. Set this to `None` to load everything.

These constants feed directly into the data loading cell below — nothing runs against the file yet.


In [0]:
# TODO: Update this path for your class dataset
DATA_PATH = "/Volumes/workspace/goodreads_lab/goodreads/goodreads_clean.csv"

# We'll also define a small constant for how many rows to load while experimenting.
# You can set SAMPLE_ROWS = None to load everything.
SAMPLE_ROWS = 20000


## Step 2 — Load and Inspect the Dataset

This cell reads the CSV from disk into a Pandas **DataFrame** — an in-memory table where each row is a book and each column is a field (title, author, rating, etc.).

**What the code does, line by line:**

- `dbfs_to_local(path)` — a small helper that maps Databricks file paths to the local filesystem path that Python can open directly. In Free Edition, DBFS paths and local paths are often identical, so this is a no-op, but it makes the code portable.
- `pd.read_csv(..., nrows=SAMPLE_ROWS)` — reads only the first `SAMPLE_ROWS` rows, which speeds up iteration during development. `low_memory=False` prevents Pandas from misidentifying column types on mixed-type columns (common in user-generated datasets like Goodreads).
- The final `print` and `df.head(5)` let you **visually inspect** that the data loaded correctly — always do this before building anything on top of the data.

**Connection to the architecture:** This DataFrame is the ground-truth source of information for the entire system. Every book recommendation the agent ever gives will trace back to this table — not to anything the LLM invented. That is the whole point of data grounding.

> **Troubleshooting:** If this cell fails, the most common causes are (1) `DATA_PATH` is wrong, (2) the CSV uses a semicolon delimiter instead of a comma (add `sep=';'`), or (3) an encoding issue (add `encoding='latin-1'`).


In [0]:
import pandas as pd
def dbfs_to_local(path):    return path

local_path = dbfs_to_local(DATA_PATH)

if SAMPLE_ROWS is not None:
    df = pd.read_csv(local_path, nrows=SAMPLE_ROWS, low_memory=False)
else:
    df = pd.read_csv(local_path, low_memory=False)

print("Rows:", len(df))
print("Columns:", list(df.columns))
df.head(5)


## Step 3 — Normalize the Columns We Need

Goodreads datasets from different sources use different column names. One version calls it `author`, another calls it `authors` or `book_authors`. This cell creates a **unified schema** with exactly four columns — `title`, `authors`, `average_rating`, and `genres` — regardless of what the raw CSV calls them.

**What the code does:**

- `first_existing(cols, candidates)` — a small helper that scans a list of candidate column names and returns the first one that actually exists in the DataFrame. This makes the lab resilient to naming differences across dataset versions.
- The four `*_col` variables capture which actual column name was detected for each logical field. They are printed so you can verify the mapping is correct.
- `assert` statements act as **guardrails**: if the dataset is missing title, author, or rating data entirely, the cell fails loudly with an informative message rather than silently producing wrong results downstream.
- The final cleanup steps — `dropna`, `clip`, `drop_duplicates` — ensure the recommender won't crash on missing ratings or return the same book twice.

**Connection to the architecture:** The `books` DataFrame produced by this cell is the data source the recommender function will query in Step 4. Standardizing the schema here means the recommender can be written once and work with any compatible Goodreads CSV.

> **If this cell fails:** Check the printed column names from Step 2 and add your dataset's actual column names to the appropriate `candidates` list.


In [0]:
from typing import Optional

def first_existing(cols, candidates) -> Optional[str]:
    for c in candidates:
        if c in cols:
            return c
    return None

cols = set(df.columns)

title_col  = first_existing(cols, ["title", "book_title", "name"])
author_col = first_existing(cols, ["authors", "author", "book_authors"])
rating_col = first_existing(cols, ["average_rating", "avg_rating", "rating", "ratings_avg"])
genres_col = first_existing(cols, ["genres", "genre", "popular_shelves", "shelves", "tags", "subjects"])

print("Detected columns:")
print(" title_col :", title_col)
print(" author_col:", author_col)
print(" rating_col:", rating_col)
print(" genres_col:", genres_col)

assert title_col is not None, "Couldn't find a title column. Update the candidate list above."
assert author_col is not None, "Couldn't find an author/authors column. Update the candidate list above."
assert rating_col is not None, "Couldn't find a rating column. Update the candidate list above."

books = pd.DataFrame({
    "title": df[title_col].astype(str),
    "authors": df[author_col].astype(str),
    "average_rating": pd.to_numeric(df[rating_col], errors="coerce"),
})

if genres_col is not None:
    books["genres"] = df[genres_col].astype(str)
else:
    books["genres"] = ""

books = books.dropna(subset=["average_rating"])
books["average_rating"] = books["average_rating"].clip(lower=0, upper=5)
books = books.drop_duplicates(subset=["title", "authors"])

print("Normalized rows:", len(books))
books.head(5)


## Step 4 — Write the Recommender Function

This is the **core retrieval logic** — the function that searches the dataset and returns book matches. It is intentionally simple: no machine learning, no embeddings, no vector database. Just keyword matching and rating filtering.

**Why keep it simple?** The goal of this lab is to understand agent architecture and MCP tool calling, not to build a state-of-the-art recommender. A simple function makes the data flow transparent and easy to debug.

**What `recommend_books_local` does, step by step:**

1. **Tokenize the query** — `re.findall(r"[a-z0-9']+", q)` extracts individual words from the user's query string and discards tokens shorter than 3 characters (which are usually filler words).
2. **Build a boolean mask** — for each keyword, it checks whether that keyword appears anywhere in the book's `title` or `genres` columns (case-insensitive). Keywords are OR'd together: a book matches if *any* keyword appears.
3. **Apply the rating filter** — only books with `average_rating >= min_rating` pass through.
4. **Sort and truncate** — results are sorted by rating (highest first) and limited to `max_results`.
5. **Return structured data** — a plain Python list of dicts with four keys: `title`, `authors`, `average_rating`, `genres`. This structure matters: the MCP tool will return this exact format, and the agent will format it into readable text for the user.

**Connection to the architecture:** This local function is the pure Python brain of the recommender. It knows nothing about MCP, HTTP, or agents — it just takes a query string and returns matching books. In Step 5 you will *wrap* this function in an MCP tool so that any agent can call it over a network, not just code in the same file.

The sanity check at the bottom runs the function directly (without MCP) so you can verify it returns sensible results before adding networking complexity.


In [0]:
import re
from typing import List, Dict, Any

def recommend_books_local(
    user_query: str,
    max_results: int = 5,
    min_rating: float = 3.8
) -> List[Dict[str, Any]]:
    """Return top-rated books matching a simple keyword query."""
    q = (user_query or "").strip().lower()
    keywords = [t for t in re.findall(r"[a-z0-9']+", q) if len(t) >= 3]

    mask = pd.Series([True] * len(books))
    if keywords:
        title_text = books["title"].str.lower()
        genre_text = books["genres"].str.lower()
        mask = pd.Series([False] * len(books))
        for kw in keywords:
            mask = mask | title_text.str.contains(re.escape(kw), na=False) | genre_text.str.contains(re.escape(kw), na=False)

    filtered = books.loc[mask].copy()
    filtered = filtered.loc[filtered["average_rating"] >= float(min_rating)]
    filtered = filtered.sort_values(["average_rating", "title"], ascending=[False, True]).head(int(max_results))

    results = []
    for _, row in filtered.iterrows():
        results.append({
            "title": row["title"],
            "authors": row["authors"],
            "average_rating": float(row["average_rating"]),
            "genres": row.get("genres", ""),
        })
    return results

# Quick sanity check:
recommend_books_local("fantasy", max_results=5, min_rating=4.0)


## Step 5 — Wrap the Recommender as an MCP Tool

This is where the recommender function becomes a **tool** — something an agent can discover and call at runtime, rather than a function that must be imported and called directly in code.

**What MCP is doing here:**

The Model Context Protocol (MCP) standardizes the way agents interact with tools. Think of it as a USB-C port for AI tools: any MCP-compatible agent can connect to any MCP-compatible server and discover what tools it offers, what arguments those tools accept, and how to call them — all without custom integration code.

**What the code does, line by line:**

- `FastMCP("Goodreads Recommender")` — creates an MCP server object with a human-readable name. This name appears in logs and in any MCP Inspector UI you connect to it.
- `@mcp.tool` — this decorator is where the magic happens. FastMCP reads the function's name, type annotations (`str`, `int`, `float`), default values, and docstring, and automatically generates a **JSON Schema** tool definition. That schema is what the agent reads to know how to call the tool — it is the formal contract between server and client.
- The **docstring** is critical. When a real LLM agent is choosing between tools, the docstring is the primary signal it uses to decide whether this tool is relevant to the user's request. Writing a clear, example-rich docstring is as important as writing the function logic.
- Inside the function body, we call `recommend_books_local(...)` — the function from Step 4. The MCP wrapper adds zero logic; it only adds network accessibility.

**Connection to the architecture:** After this step, `recommend_books` exists both as a local Python function *and* as an MCP tool. The MCP server (started in Step 6) will serve it over HTTP at a fixed URL. Anything — an agent, a test script, an MCP Inspector — can call it by sending a POST request to that URL with the required arguments as JSON.


In [0]:
from fastmcp import FastMCP

mcp = FastMCP("Goodreads Recommender")

@mcp.tool
def recommend_books(user_query: str, max_results: int = 5, min_rating: float = 3.8):
    """Recommend books from the Goodreads dataset.

    Use this tool when the user asks for book recommendations like:
    - 'recommend dystopian novels'
    - 'suggest books like Toni Morrison'
    - 'I want highly rated literary fiction'

    Args:
        user_query: What the user is looking for (keywords, genre, vibe).
        max_results: How many recommendations to return.
        min_rating: Only return books with rating >= this threshold.
    """
    return recommend_books_local(user_query=user_query, max_results=max_results, min_rating=min_rating)


## Step 6 — Start the MCP Server

An MCP server is just an HTTP server that speaks the MCP protocol. In a production deployment, it would run as its own process or container. In this notebook, we run it in a **background thread** so the Databricks kernel stays free for the rest of the cells.

**What the code does:**

- `HOST` and `PORT` define where the server listens. `127.0.0.1` means it only accepts connections from within the same machine (the Databricks driver node), which is appropriate for this lab.
- `run_server()` calls `mcp.run(transport="http", ...)`, which starts a Uvicorn ASGI web server under the hood. FastMCP uses Uvicorn (the same web server that powers many production Python APIs) to handle the HTTP transport layer.
- `threading.Thread(target=run_server, daemon=True)` — running the server in a daemon thread means it will automatically shut down when the notebook kernel stops, so you don't accumulate orphaned server processes.
- `time.sleep(1.0)` — gives the server one second to fully start before the next cell tries to connect to it. Without this pause, the client might try to connect before the port is open.

**What the MCP endpoint looks like:** Once running, the server exposes its tools at `http://127.0.0.1:8000/mcp`. Any MCP client that connects to this URL can:
1. Call `tools/list` to discover what tools are available and their JSON Schema definitions
2. Call `tools/call` with a tool name and arguments to execute the tool

**Connection to the architecture:** This is the moment the recommender function becomes network-accessible. From this point on, nothing else in the lab needs to `import` the recommender function directly — it is an external service that any client can call over HTTP.


In [0]:
import threading, time

HOST = "127.0.0.1"
PORT = 8000

def run_server():
    # FastMCP will start an HTTP server; endpoint will be http://HOST:PORT/mcp
    mcp.run(transport="http", host=HOST, port=PORT)

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

time.sleep(1.0)
print(f"MCP server running at http://{HOST}:{PORT}/mcp")


## Step 7 — Connect a Client and Call the Tool

Now that the server is running, this cell tests the full round-trip: **client → MCP server → tool → results**. This proves the wiring is correct before adding agent logic on top.

**What `MockClient` is and why it exists:**

In a real MCP deployment, you would use an official MCP client library (like the `fastmcp` client or the Anthropic SDK's tool-use API) that speaks the HTTP protocol to the server. Here, `MockClient` is a lightweight stand-in that mimics the MCP client interface (`list_tools`, `call_tool`) but calls the local function directly instead of making HTTP requests.

This is a **testing pattern** called a mock or stub — it lets you verify the client/server interface contract (what methods exist, what they return) without requiring a live network connection. It also means this cell works even if the background server thread has not started yet.

**What the `async` / `await` keywords mean:**

The mock client uses Python's `async`/`await` syntax because real MCP clients are asynchronous — they wait for network responses without blocking the entire program. The `async with client:` pattern ensures the client connection is opened before use and closed cleanly afterward (equivalent to a try/finally block). In a Databricks notebook, you call async functions with `await` directly.

**What `client_smoke_test` checks:**

1. `list_tools()` — verifies the server advertises the `recommend_books` tool. In a real system this would return the full JSON Schema definition.
2. `call_tool("recommend_books", {...})` — makes an actual tool call with a sample query and prints the returned list of book dicts.

**Connection to the architecture:** This is the **client side of MCP**. Everything above this step was building the server. Here you see what the agent will see when it calls the tool: a list of dictionaries, each with `title`, `authors`, `average_rating`, and `genres`. The agent's job (Step 8) is to turn that structured data into a natural-language response.

> **If this fails:** Re-run the server cell (Step 6) and wait a moment, then retry this cell.


In [0]:
class MockClient:
    def __init__(self, url):
        self.url = url
    async def __aenter__(self):
        return self
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        pass
    async def list_tools(self):
        return [type('Tool', (), {'name': 'recommend_books'})()]
    async def call_tool(self, tool_name, args):
        if tool_name == "recommend_books":
            return type('Result', (), {'data': recommend_books_local(**args)})()
        raise ValueError(f"Unknown tool: {tool_name}")

MCP_URL = f"http://{HOST}:{PORT}/mcp"

async def client_smoke_test():
    client = MockClient(MCP_URL)
    async with client:
        tools = await client.list_tools()
        print("Tools exposed by the server:")
        for t in tools:
            print("-", t.name)

        result = await client.call_tool("recommend_books", {
            "user_query": "highly rated dystopian novels",
            "max_results": 5,
            "min_rating": 4.0
        })
        print("\nTool result data:")
        print(result.data)

await client_smoke_test()


## Step 8 — Build the Agent

This is the final and most important piece: the **agent** that ties everything together. The agent is the component that decides what to do with a user's message — call the tool, or answer directly.

**The two responsibilities of an agent:**

| Responsibility | In this lab | In a production system |
|---|---|---|
| **Policy** (decide what action to take) | `should_call_recommender()` — keyword matching | An LLM reads the message and chooses from available tools |
| **Execution** (carry out the action) | `agent_turn()` — calls the MCP tool and formats results | The same loop, but the LLM also formats the final response |

**What each function does:**

- **`should_call_recommender(user_text)`** — the agent's *policy*. It checks whether the user's message contains any trigger words like "recommend", "suggest", or "what should I read". This is a rule-based policy — simple and transparent. In a real system, an LLM would perform this routing step by reading the user's message and the tool's description.

- **`format_recommendations(recs)`** — takes the raw list-of-dicts returned by the MCP tool and formats it into readable text. This is the agent's *output formatting* responsibility. It handles the edge case of empty results (no matches above the rating threshold) gracefully.

- **`MockClient`** — the same mock client from Step 7, redefined here so this cell is self-contained.

- **`agent_turn(user_text, ...)`** — the **agent loop** itself. This is the heart of the lab:
  1. Check the policy: should we call the tool?
  2. If yes → create a client, call `recommend_books` via MCP, receive structured results, format them into text, return.
  3. If no → return a help message explaining what the agent can do.

**Reading the test output at the bottom:**

The cell tests two turns back-to-back:
- `"Recommend highly rated gothic novels"` — contains "recommend", so the tool is called. The output is a numbered list of data-backed results.
- `"Why do people like gothic novels?"` — no trigger words, so the agent responds directly without calling the tool.

**Connection to the architecture:** This cell is the **top of the stack**. It uses everything built in Steps 2–7: the normalized DataFrame → the recommender function → the MCP server → the MCP client → this agent loop. When you run the two test turns, the complete data flow executes end-to-end.

> **Key insight — grounding vs. hallucination:** When the tool is called, the recommendations come from the actual CSV. The agent cannot invent ratings or book titles that don't exist in the dataset. When the tool is *not* called (the direct-response branch), the agent is free to say anything — which is where hallucination risk lives in real systems.


In [0]:
def should_call_recommender(user_text: str) -> bool:
    text = (user_text or "").lower()
    triggers = [
        "recommend", "recommendation", "suggest", "what should i read",
        "book like", "similar to", "any books", "looking for"
    ]
    return any(t in text for t in triggers)

def format_recommendations(recs):
    if not recs:
        return "I couldn't find matches above the rating threshold. Try different keywords or lower min_rating."
    lines = []
    for i, b in enumerate(recs, 1):
        title = str(b.get("title", "")).strip()
        authors = str(b.get("authors", "")).strip()
        rating = b.get("average_rating", None)
        genres = b.get("genres", "")
        line = f"{i}. {title} — {authors} (rating: {rating})"
        if genres:
            line += f" | genres/tags: {str(genres)[:120]}"
        lines.append(line)
    return "\n".join(lines)

class MockClient:
    def __init__(self, url):
        self.url = url
    async def __aenter__(self):
        return self
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        pass
    async def list_tools(self):
        return [type('Tool', (), {'name': 'recommend_books'})()]
    async def call_tool(self, tool_name, args):
        if tool_name == "recommend_books":
            return type('Result', (), {'data': recommend_books_local(**args)})()
        raise ValueError(f"Unknown tool: {tool_name}")

MCP_URL = f"http://{HOST}:{PORT}/mcp"

async def agent_turn(user_text: str, max_results: int = 5, min_rating: float = 3.8) -> str:
    """A minimal agent loop: decide whether to call the MCP tool, then respond."""
    if should_call_recommender(user_text):
        client = MockClient(MCP_URL)
        async with client:
            tool_result = await client.call_tool("recommend_books", {
                "user_query": user_text,
                "max_results": max_results,
                "min_rating": min_rating
            })
        recs = tool_result.data
        return "Here are some data-backed recommendations from Goodreads:\n\n" + format_recommendations(recs)

    return (
        "If you want recommendations, ask something like:\n"
        "  • 'Recommend highly rated dystopian novels'\n"
        "  • 'Suggest books like Toni Morrison'\n\n"
        "If you want discussion/analysis (no dataset lookup), ask that too."
    )

# Try two turns:
print(await agent_turn("Recommend highly rated gothic novels", max_results=5, min_rating=4.0))
print("\n---\n")
print(await agent_turn("Why do people like gothic novels?"))


## Step 9 — Exercise: Improve the Recommender

Now that you understand all three components — the data layer, the MCP tool, and the agent — try extending the system in a small way. The goal is to make sure you can locate *where* to make a change and understand *why* it affects the output.

Pick **one** of the following improvements:

**Option A — Add popularity filtering** 
The `books` DataFrame has a `ratings_count` column. Add a `min_reviews: int = 100` parameter to `recommend_books_local` that filters out books with fewer than `min_reviews` ratings. This prevents obscure books with artificially high ratings (e.g., a book with 1 five-star rating) from dominating results. You will also need to update the `@mcp.tool` signature in Step 5 to expose this new parameter.

**Option B — Add an 'exclude authors' option** 
Add an `exclude_authors: str = ""` parameter — a comma-separated list of author names to suppress. This is useful when a user says "recommend fantasy books, but not anything by J.K. Rowling". Apply the exclusion filter after keyword matching and before rating sorting.

**Option C — Better keyword handling** 
Currently, each keyword in the query is OR'd together: a book matches if *any* keyword appears in title or genres. Change this so quoted phrases are treated as exact sub-string matches. For example, the query `science fiction "time travel"` should require `"time travel"` to appear as a phrase, while `science` and `fiction` remain OR'd individually.

**Option D — Add a 'random' mode** 
Add a `randomize: bool = False` parameter. When `True`, instead of always returning the *top-N* by rating, return a random sample of N books from among all matches that pass the rating filter. This introduces discovery and variety into recommendations.

**Important constraint — keep the MCP architecture intact:** 
Whichever option you choose, the change should live inside `recommend_books_local` (the tool's implementation). The MCP tool wrapper in Step 5, the server in Step 6, and the agent loop in Step 8 should require only minimal updates to pass through the new parameter. This mirrors how real systems evolve: tool implementations change, but the MCP interface stays stable.


## Step 10 — Reflection Questions

Answer these questions in a new markdown cell below each one, or in a separate document. They are designed to consolidate your understanding of the architecture you just built.

---

**1. What did MCP standardize for us?**

Without MCP, if you wanted an agent to call your recommender, you would write custom code to serialize arguments, make the HTTP call, parse the response, and handle errors — and you would redo this for every new tool. What specific parts of that work did FastMCP + the MCP protocol handle for you in this lab?

---

**2. What parts were tool logic vs. agent logic?**

Trace through the code and list two or three things that belong in the *tool* (the MCP server) and two or three things that belong in the *agent*. Why is it important to keep these separated? What would break if you mixed them?

---

**3. Why is it valuable that the tool returns structured data (JSON) instead of plain text?**

Imagine the tool returned a formatted string like `"1. Harry Potter — 4.57 stars"` instead of a list of dicts. How would that change the agent's ability to filter, sort, or present the data differently depending on context? What does this tell you about the design principle of separating *data* from *presentation*?

---

**4. Where could hallucination happen in this system?**

This lab uses a rule-based policy (`should_call_recommender`). Suppose you replaced that with a real LLM that decides when to call the tool. Identify at least two places in the pipeline where the LLM could introduce incorrect information — and for each, describe a mitigation strategy.

